<a href="https://colab.research.google.com/github/bgalerne/mva_generative_models_for_images/blob/main/2_mva_gm_generative_prior_for_inverse_problems.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Generative prior for imaging inverse problems


**Author:**

Bruno Galerne: www.idpoisson.fr/galerne / https://github.com/bgalerne

## Introduction

In this notebook we will use a generative model as an image prior to solve an imaging inverse problem.
This amounts to limit the space of images to the subset
$$
\{x = G(z),~z\in\mathbb{R}^k\} \subset \mathbb{R}^d
$$
to solve a least squares inverse problem
$$
\min_{x} \|\mathcal{A}(x) - y \|^2.
$$

We will consider an ill-posed inverse problem close to tomography reconstruction: Reconstruct an image given the sum of its gray-levels along four directions.


## Generative network

We will consider images from the **MNIST test set** and a generative model trained as a GAN using a the disjoint **MNIST training set**.

The generator is set to generate images in the range $[-1,1]$ but the real data is in the range $[0,1]$.
Before applying the operator $\mathcal{A}$ one must come back to the original data range (ie one does not measure negative values).

In [ ]:
from __future__ import print_function

import matplotlib.pyplot as plt
import numpy as np

import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
import torchvision
from torchvision import datasets, transforms
from torch.optim.lr_scheduler import StepLR
import torch.autograd as autograd
import os

from PIL import Image
from IPython.display import display

# Device configuration
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
print(device)

In [ ]:
def stretch(x):
  # stretch values such that [min. max]->[0,1]
  m = torch.min(x)
  M = torch.max(x)
  if M>m:
    return((x-m)/(M-m))
  else:
    return(torch.zeros(x.shape))

def imshow(img, unnormalize=True, zoom_factor=3, stretch_opt=False):
    img = img.clone().detach().to('cpu')
    if unnormalize:
      img = img*0.5 + 0.5     # unnormalize
    if stretch_opt:
      img = stretch(img)
    if zoom_factor!=1:
      img = torch.kron(img, torch.ones(1,zoom_factor,zoom_factor))
    pil_img = torchvision.transforms.functional.to_pil_image(img)
    display(pil_img)
    return(pil_img)


### Load pretrained generative network:

In [ ]:
!wget -c 'https://www.idpoisson.fr/galerne/mva/GAN_G_net_ep100.pth'

In [ ]:
# Generator network:
class G_Net(nn.Module):
  def __init__(self, k):
    super(G_Net, self).__init__()
    self.fc1 = nn.Linear(k, 256)
    self.fc2 = nn.Linear(256, 512)
    self.fc3 = nn.Linear(512, 784)

  def forward(self,x):
    x = self.fc1(x)
    x = F.leaky_relu(x, negative_slope=0.2)
    x = self.fc2(x)
    x = F.leaky_relu(x, negative_slope=0.2)
    x = self.fc3(x)
    x = torch.tanh(x)
    x = x.view(-1,1,28,28) # batch_size x channels x H x W
    return(x)

# initialize generator (with random weights)
k = 32
G_net = G_Net(k).to(device)

def show_G_net(z=None):
  # provide random latent code as option to see evolution
  with torch.no_grad():
    if z==None:
      z = torch.randn(100,k).to(device)
    genimages = G_net(z)
    pil_img = imshow(torchvision.utils.make_grid(genimages.to('cpu'),nrow=10))
    return(pil_img)
    #print(disnet(genimages))
print("Generator with random init:")
show_G_net();

G_net.load_state_dict(torch.load('GAN_G_net_ep100.pth', map_location=device))
G_net.eval()
G_net.requires_grad_(False)
print("Pretrained generator:")
show_G_net(z);


## Operator of inverse problem

We will first define the linear operator $\mathcal{A}$ as ```opA```.
We will consider the operator that sums the values of the gray-level image along each vertical, horizontal and diagonal directions.

In [ ]:
# We suppose that:
#  - the input x is a square gray-level image of size 1xMxN with M=N
#  - the output y is 1D tensor of size number of measurements m

def sum_all_diagonal_matrix(mat: torch.tensor): 
  # from: https://stackoverflow.com/questions/57347896/sum-all-diagonals-in-feature-maps-in-parallel-in-pytorch
  n,_ = mat.shape
  zero_mat = torch.zeros((n, n),device=mat.device) # Zero matrix used for padding
  mat_padded =  torch.cat((zero_mat, mat, zero_mat), 1) # pads the matrix on left and right
  mat_strided = mat_padded.as_strided((n, 2*n), (3*n + 1, 1)) # Change the strides
  sum_diags = torch.sum(mat_strided, 0) # Sums the resulting matrix's columns
  return(sum_diags[1:])

def axial_and_diagonal_sum(x):
  # sum over diagoanal:
  _,M,N = x.shape
  xmat = x.reshape(M,N)
  yhori = torch.sum(x, axis=2).flatten()
  yvert = torch.sum(x, axis=1).flatten()
  ydiag = sum_all_diagonal_matrix(xmat).flatten()
  y_anti_diag = sum_all_diagonal_matrix(xmat.flip(1)).flatten()
  y = torch.cat((yhori, yvert, ydiag, y_anti_diag.flip(0)))
  return(y)

# test of axial_and_diagonal_sum(x)
t = torch.diag(1+torch.arange(4)).unsqueeze(0)
print("Test of axial_and_diagonal_sum(x):")
print("Input:", t)
print("Output:", axial_and_diagonal_sum(t))      

opA = axial_and_diagonal_sum

def batchopA(x):
  # apply opA to each image of a batch and return a tensor:
  listAx = []
  for bidx in range(x.shape[0]):
    listAx.append(opA(x[bidx,:,:]))
  Ax = torch.stack(listAx)
  return(Ax)


## Input data

In [ ]:
# input
transformtest = transforms.ToTensor()
datatest = datasets.MNIST('.', train=False, download=True, transform=transformtest)
input_idx = 0 # a seven
#input_idx = 99 # a nine
#input_idx = 3 # a zero
#input_idx = 5 # a one
#input_idx = 21 # a six
#input_idx = 1 # a two
#input_idx = 4 # a four
#input_idx = 1984 # an unusual 2, hard


xstar_im =  datatest[input_idx][0].to(device)
imshow(xstar_im, False);
_, M, N = xstar_im.shape
d = M*N

y = opA(xstar_im)
noise = 0.5*torch.randn(y.shape, device=device)
ynoisy = y+noise

print('Plot of y: (yhori, yvert, ydiag, y_anti_diag)')
plt.figure(figsize=(20,4))
plt.bar(range(y.numel()), y.to('cpu').numpy());


## Pseudo-inverse of operator

Let us recall that the Moore-Penrose pseudo-inverse $\mathcal{A}^+$ is such that $\mathcal{A}^+ y$ is a least-square solution with minimal norm
$$
\min_{x\in\mathbb{R}^{d}} \|\mathcal{A}(x) - y \|^2 \quad \text{s.t.}\quad\text{$\|x\|$ is minimal}.
$$
One can compute $\mathcal{A}^+ y$ by applying gradient descent to the convex function
$$
f(x) = \|\mathcal{A}(x) - y \|^2
$$
(as long as the descent step is smaller than $1/L$ the Lipschitz constant of the gradient).


In [ ]:
x = torch.zeros(xstar_im.shape).to(device)
x.requires_grad = True

# compute pseudo-inverse of y
optimizer = optim.SGD([x], lr = 0.01, momentum = 0.9)
niter = 1000
for it in range(niter):
  optimizer.zero_grad()
  fx = torch.sum((opA(x)-y)**2)
  fx.backward()
  optimizer.step()
  if fx.item()<1e-10:
    print("Convergence reached:")
    print("iteration ", it, "fx = ", fx.item())
    imshow(x, stretch_opt=True);
    break
  if it%(niter//10) == niter//10-1:
    print("iteration ", it, "fx = ", fx.item())
    imshow(x, stretch_opt=True);


## GAN prior for solving the inverse problem
We solve for
$$
\hat x = G(\hat z)
\quad \text{with $\hat z$ solution of}\quad 
\min_{z\in\mathbb{R}^{k}} \|\mathcal{A}(G(z)) - y \|^2.
$$
Since the result highly depends on the initialization, we will optimize for a batch of $b=10$ $z$ values.

**Exercise:**
Implement a script that:
1. Initialize an array $z = (z^0, \dots, z^{b-1})$ of $b$ random latent code.
2. Optimize for $Z$ to minimize the sum
$$
\sum_{j=0}^{b-1} \|\mathcal{A}(G(z^j)) - y \|^2
$$
using ```optim.Adam([z], lr = 0.01)``` as optimizer for ```niter = 10**4```.
3. Display the $b=10$ corresponding images $G(z^j)$ at initialization and at 10 intermediary steps as well as the iteration number and the value of the function to optimize.

Be carefull that $\mathcal{A}$ should be applied to images with range $[0,1]$.


In [ ]:
b = 10 # batch size:
z = torch.rand(b,k, device=device)
z.requires_grad = True

optimizer = optim.Adam([z], lr = 0.01)

print('Solution:')
imshow(xstar_im, False);
niter = 10**4
for it in range(niter):

  #TODO

  
  if it==0 or it%(niter//10) == niter//10-1:
    print("iteration ", it, "fx = ", fx.item())
    show_G_net(z=z)



## Sensitivity to noise

**Exercise:** Experiment if the pseudo-inverse and the output obtained with generative prior are sensible with respect to noisy measurements.

## Sensitivity to generator

**Exercise:**
1. Evaluate the FID score of the generator that is approximate.
2. Find a pretrained generative model online with better FID score and use it to improve the results (but make sure that the network is deterministic in evaluation mode).